In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [21]:
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature
import keras

2025-05-03 15:14:37.708046: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-03 15:14:37.988478: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 15:14:39.665438: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 15:14:40.673846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746263681.519590 1502352 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746263681.73

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/media/saif/New Volume/GenomeArcDev/MLOps/MLops/venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.4 is exactly one major version older than the runtime version 6.30.2 at google/protobuf/any.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/media/saif/New Volume/GenomeArcDev/MLOps/MLops/venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.4 is exactly one major version older than the runtime version 6.30.2 at google/protobuf/wrappers.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [10]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")

In [11]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [13]:
## split the data into training, validation and test set
train,test = train_test_split(data,test_size=0.25,random_state=42)

In [15]:
train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel()

In [16]:
test_x = test.drop(['quality'],axis=1).values

In [17]:
test_y = test[['quality']].values.ravel()

In [42]:
train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.20, random_state=42)

In [20]:
signature = infer_signature(train_x,train_y)

In [56]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [28]:
def objective(params):
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y

    )

    return result

In [30]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [57]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/37 ━━━━━━━━━━━━━━━━━━━━ 10s 299ms/step - loss: 32.2701 - root_mean_squared_error: 5.6807
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 32.1812 - root_mean_squared_error: 5.6723 - val_loss: 26.5249 - val_root_mean_squared_error: 5.1502

Epoch 2/3                                            

 1/37 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - loss: 26.5883 - root_mean_squared_error: 5.1564
13/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 26.5744 - root_mean_squared_error: 5.1549 
24/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 25.9801 - root_mean_squared_error: 5.0966
35/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 25.4744 - root_mean_squared_error: 5.0463
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 25.3437 - root_mean_squared_error: 5.0332 - val_loss: 20.5596 - val_root_mean_squared_error: 4.5343

Epoch 3/3                                            

 1/37 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 21.9928 - root_mean_squared_error: 4.6896

In [60]:
import mlflow

model_uri = 'runs:/7a4e1b77a55443ca828507a7d311a8b0/model'

# Replace INPUT_EXAMPLE with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
input_data = test_x

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="local",
)

2025/05/03 16:23:43 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.
2025/05/03 16:23:43 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
{"predictions": [[5.7780303955078125], [7.090057373046875], [6.612865447998047], [5.324977874755859], [6.124493598937988], [6.498795032501221], [5.807153701782227], [5.49308967590332], [6.233182907104492], [5.921135902404785], [7.090582370758057], [4.616716384887695], [6.741005897521973], [5.153820037841797], [7.705026626586914], [5.110430717468262], [7.34281063079834], [6.249259948730469], [6.505252838134766], [5.227218151092529], [5.363917350769043], [6.506011009216309], [4.95101261138916], [5.729887962341309], [5.7222442626953125], [5.133843421936035], [4.728233337402344], [6.220978260040283], [5.614713668823242], [5.223491191864014], [5.472538948059082], [5.444295883178711], [6.142139911651611], [5.193694114685059], [5.494067668914795], [6.843197822570801], [6.930771827697754], [4.872823715209961], [5.26069974899292], [5.958134174346924], [5.835751533508301], [5.318768501281738], [5.570821285247803], [5.9979448318481445], [4.911815166473389], 

In [63]:
import mlflow
model_uri = 'runs:/7a4e1b77a55443ca828507a7d311a8b0/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[5.7780304],
       [7.0900574],
       [6.6128654],
       ...,
       [6.398208 ],
       [7.093894 ],
       [5.414186 ]], dtype=float32)

In [65]:
## Register in the model registry
mlflow.register_model(model_uri,"Wine-Quality")

Successfully registered model 'Wine-Quality'.
Created version '1' of model 'Wine-Quality'.


<ModelVersion: aliases=[], creation_timestamp=1746267949943, current_stage='None', description=None, last_updated_timestamp=1746267949943, name='Wine-Quality', run_id='7a4e1b77a55443ca828507a7d311a8b0', run_link=None, source='file:///media/saif/New%20Volume/GenomeArcDev/MLOps/MLops/MLflowStarter/2-MLproject/mlruns/195242204327695645/7a4e1b77a55443ca828507a7d311a8b0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>